In [40]:
using LinearAlgebra
using ForwardDiff
const FD = ForwardDiff

ForwardDiff

In [41]:
function dynamics(x, u)
    # euler's with added nonlinearities
    J = Diagonal([1, 2, 3])
    ẋ = J \ (u * sin(u[1]) - cross(x, J * x)) * cos(u[2])
end

dynamics (generic function with 1 method)

In [42]:
function rk4(x,u,dt)
    k1 = dt * dynamics(x,        u)
    k2 = dt * dynamics(x + k1/2, u)
    k3 = dt * dynamics(x + k2/2, u)
    k4 = dt * dynamics(x + k3,   u)
    x + (1 / 6) * (k1 + 2 * k2 + 2 * k3 + k4)
end

rk4 (generic function with 1 method)

In [43]:
function dynamics_jacobians(x,u)
    A = FD.jacobian(_x -> dynamics(_x, u), x)
    B = FD.jacobian(_u -> dynamics(x, _u), u)
    return A, B
end
function dynamics_and_jacobians(x,u)
    ẋ = dynamics(x, u)
    A, B = dynamics_jacobians(x, u)
    return ẋ, A, B
end

dynamics_and_jacobians (generic function with 1 method)

In [46]:
function rk4step_jacobians(x,u,dt)

    # normal RK4 but we store the A's and B's at each evaluate point
    ẋ1, A1, B1 = dynamics_and_jacobians(x,                u)

    ẋ2, A2, B2 = dynamics_and_jacobians(x + .5 * dt * ẋ1, u)

    ẋ3, A3, B3 = dynamics_and_jacobians(x + .5 * dt * ẋ2, u)

    ẋ4, A4, B4 = dynamics_and_jacobians(x + dt * ẋ3,      u)

    x₊ = x + (1 / 6) * dt * (ẋ1 + 2 * ẋ2 + 2 * ẋ3 + ẋ4)

    # A_d
    dk1_dx1 = dt*A1
    dx2_dx1 = I + .5*dk1_dx1
    dk2_dx1 = dt*A2*dx2_dx1
    dx3_dx1 = I + .5*dk2_dx1
    dk3_dx1 = dt*A3*dx3_dx1
    dx4_dx1 = I + dk3_dx1
    dk4_dx1 = dt*A4*dx4_dx1
    A_d = I + (1/6)*(dk1_dx1 + 2*dk2_dx1 + 2*dk3_dx1 + dk4_dx1)

    # B_d
    dk1_du = dt*B1
    dx2_du = .5*dk1_du
    dk2_du = dt*A2*dx2_du + dt*B2
    dx3_du = .5*dk2_du
    dk3_du = dt*A3*dx3_du + dt*B3
    dx4_du = dk3_du
    dk4_du = dt*A4*dx4_du + dt*B4
    B_d = (1/6)*(dk1_du + 2*dk2_du + 2*dk3_du + dk4_du)

    return x₊, A_d, B_d
end

rk4step_jacobians (generic function with 1 method)

In [47]:
let 
    # random state and control 
    x = randn(3)
    u = randn(3)
    dt = 0.1 
    
    # get RK4 jacobians with ForwardDiff
    A_rk4 = FD.jacobian(_x -> rk4(_x,u,dt),x)
    B_rk4 = FD.jacobian(_u -> rk4(x,_u,dt),u)
    x2 = rk4(x,u,dt)
    
    # get RK4 jacobians with our new function
    x₊, A_d, B_d = rk4step_jacobians(x,u,dt)
    
    # show they are the same
    @show norm(x2 - x₊)
    @show norm(A_d - A_rk4)
    @show norm(B_d - B_rk4)
end

norm(x2 - x₊) = 1.734723475976807e-18
norm(A_d - A_rk4) = 9.579489523762355e-19
norm(B_d - B_rk4) = 1.5899003299612593e-17


1.5899003299612593e-17